In [1]:
def adj_dataframe(df):
    """
    Ensures that a dataframes columns are consistent for merging and for sql datatypes
    Parameters
    ---------
    df : dataframe
        a dataframe whose columns you want to adjust
    Returns
    df : dataframe
        a dataframe with adjusted columns
    -------
    """
    def convert_id(row):
        try:
            new_id = int(float(row))
        except:
            new_id = row
        return new_id

    import numpy as np
    import pandas as pd
    df = df.replace('NaN', np.nan)
    df = df.fillna(value=np.nan)
    for col in df.columns:
        if col in ['asof_date', 'Date', 'date', 'dates']:
            df[col] = pd.to_datetime(df[col])
        if col in ['id', 'external_id', 'fundId', 'Fund_ID', 'Index_ID', 'external_strategy_id', 'strategy_code', 'strategy_fund_id', 'Fund ID', 'Firm_ID', 'Firm ID', 'ret_ts_id', 'aum_ts_id', 'id_record_number', 'risk_ts_id']:
            try:
                df[col] = df[col].apply(int)

            except:  # if there are np.nan's in the column, apply(int) fails - this is a workaround
                df[col] = df[col].astype('object')

            for index, row in df.iterrows():
                df.loc[index, col] = convert_id(df.loc[index, col])
    return df


#delete if return value new = return value old
def batch_delete(list_to_delete, table_name, delete_column_name):
    """
    Deletes a list of records from a given database table, given a column name
        Validates the data to add proper quotations based on the first item in the list
        Also checks to see how many records are being deleted and will batch delete if necessary
    Parameters
    ---------
    list_to_delete : list
        a list of items, each denoting one row, that will be deleted
        each item in the list should be one unique row
    table_name: string
        the name of the table to delete from
    delete_column_name: string
        the name of the column in <table_name> to delete the items from <list_to_delete>
    Returns
    -------
    """
    from sqlalchemy.engine import URL
    from sqlalchemy import create_engine
    from math import ceil
    from pyodbc import connect
    import pandas as pd
    connection_string = 'Driver={SQL Server};Server=scdb1.silvercreeksv.com;Database=scfundrisk;Trusted_Connection=yes;'
    connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

    engine = create_engine(connection_url)
    conn = connect(connection_string)
    cursor = conn.cursor()

    if type(list_to_delete) is not list:
        raise ValueError("""'list_to_delete' must be of type list """)
    # get initial record count
    beg_no_records_df = pd.read_sql_query("""select count("""+delete_column_name+""") as ct from """+table_name, engine)
    no_records = beg_no_records_df.loc[0, 'ct']
    if len(list_to_delete) > 0:
        # the database can only delete 2090 records in one go
        # if number of records>2090, we have to beak it up
        if len(list_to_delete) > 2090:
            print('need to batch delete to accomodate database limits')
            num_iterations = ceil(len(list_to_delete)/2090)
            for i in range(num_iterations):
                print('deleting rows '+str(i*2090)+' to '+str(min(2090*(i+1), len(list_to_delete))))
                sub_list_to_delete = list_to_delete[2090*(i):2090*(i+1)]
                if type(sub_list_to_delete[0]) == str:
                    # we only check the first element because sql ensures constant datatypes
                    sub_list_to_delete = "','".join((map(str, sub_list_to_delete)))
                    sub_list_to_delete = "'"+sub_list_to_delete+"'"
                else:
                    sub_list_to_delete = ','.join((map(str, sub_list_to_delete)))
                sub_list_to_delete = '('+sub_list_to_delete+')'
                cursor.execute(''' DELETE FROM '''+table_name+''' where '''+delete_column_name+''' in '''+sub_list_to_delete)
                conn.commit()

        elif len(list_to_delete) > 0:
            if type(list_to_delete[0]) == str:
                # we only check the first element because sql ensures constant datatypes
                temp_list_to_delete = "','".join((map(str, list_to_delete)))
                temp_list_to_delete = "'"+temp_list_to_delete+"'"
            else:
                temp_list_to_delete = ','.join((map(str, list_to_delete)))
            temp_list_to_delete = '('+temp_list_to_delete+')'
            cursor.execute(''' DELETE FROM '''+table_name+''' where '''+delete_column_name+''' in '''+temp_list_to_delete)
            conn.commit()
        # get updated record count
        end_no_records_df = pd.read_sql_query("""select count("""+delete_column_name+""") as ct from """+table_name, engine)
        end_no_records = end_no_records_df.loc[0, 'ct']
        print('deleted '+str(no_records-end_no_records)+' number of records from table: '+table_name+', based on column: ' + delete_column_name)
    else:
        print('no records to delete from: '+table_name)

In [2]:
 #Download S&P US LLI from SFTP  
import datetime  
import paramiko
import pandas as pd
import datetime
import xlrd
now = datetime.datetime.now()
year = now.strftime("%Y")
month = now.strftime("%#m")
#server = 'lcdx.pitchbook.com'
    # server_ip=3.130.0.55

ssh_client = paramiko.SSHClient()


server = 'sftp.lcdx.pitchbook.com'
port = 22
user = 'SilverCreek'
password = '4E~l#4qH'


ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh_client.connect(server,port,user,password)
ssh_client = ssh_client.open_sftp()
remotepath = '/Inbox/Returns Summary-'+month+'-' +year+'.xls'
localpath='//seanas.silvercreeksv.com/shared drive/Investments/Investment Process/Tools/Data Ingestion/LCD/Raw Data/LCDMostRecent.xls'
ssh_client.get(remotepath, localpath)

#GICS III

xls = pd.ExcelFile('//seanas.silvercreeksv.com/shared drive/Investments/Investment Process/Tools/Data Ingestion/LCD/Raw Data/LCDMostRecent.xls') 
df1 = pd.read_excel(xls,'GICS III Returns' )
# read by default 1st sheet of an excel file
df1 = df1.iloc[12:]
df2 = df1.dropna(how='all')

df3 = df2.dropna(how='all',axis=1)
df4 = df3.dropna(thresh = 10)
df4 = df4.reset_index(drop=True)
df5 = df4.rename(columns = {'Unnamed: 0': 'Date'})
df5['Date'] = pd.to_datetime(df5['Date'])
df6 = df5.sort_values(by='Date', ascending = True)
#df6.head(10)

#obtain LLI Rating Returns
dfa1 = pd.read_excel(xls, "Monthly")
dfa2 = dfa1.dropna(how='all')
dfa3 = dfa2.dropna(how='all',axis=1)
#dfa3['Date'] = pd.to_datetime(dfa3['Date'])
dfa4 = dfa3.sort_values(by='Date', ascending = True)
dfa5 = dfa4[['Date','BBB Index','BB Index', 'B Index','BB/B Index', 'CCC Only Index']]


#merge
df7 = pd.merge(df6, dfa5,left_on = 'Date',right_on = 'Date',how = 'left')
df7.columns = ['LLI- ' + str(col) for col in df7.columns]
df8LLI = df7.rename(columns={'LLI- Date': 'Date'})
#df8LLI.to_excel('//seanas.silvercreeksv.com/shared drive/Investments/Investment Process/Tools/Data Ingestion/LCD/LLI_GICS_Cleaned.xlsx', index = False)

In [3]:
#ELLI download off SFTP
import datetime  
import paramiko
    #import keyring 
now = datetime.datetime.now()
year = now.strftime("%Y")
month = now.strftime("%#m")


ssh_client = paramiko.SSHClient()

server = 'sftp.lcdx.pitchbook.com'
    # server_ip=3.130.0.55
port = 22
user = 'SilverCreek'

password='4E~l#4qH'

ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh_client.connect(server,port,user,password)
ssh_client = ssh_client.open_sftp()
remotepath = '/Inbox/ELLI Returns Summary.xlsx'
localpath='//seanas.silvercreeksv.com/shared drive/Investments/Investment Process/Tools/Data Ingestion/LCD/Raw Data/ELLIMostRecent.xls'
ssh_client.get(remotepath, localpath)

#ELLI Clean and Export
import pandas as pd
import datetime
xls = pd.ExcelFile('//seanas.silvercreeksv.com/shared drive/Investments/Investment Process/Tools/Data Ingestion/LCD/Raw Data/ELLIMostRecent.xls') 
df1 = pd.read_excel(xls,'GICS III Returns' )
# read by default 1st sheet of an excel file
df1 = df1.iloc[12:]
df2 = df1.dropna(how='all')

df3 = df2.dropna(how='all',axis=1)
df4 = df3.dropna(thresh = 10)
df4 = df4.reset_index(drop=True)
df5 = df4.rename(columns = {'Unnamed: 0': 'Date'})
df5['Date'] = pd.to_datetime(df5['Date'])
df6 = df5.sort_values(by='Date', ascending = True)


#obtain ELLI Rating Returns
dfa1 = pd.read_excel(xls, "Monthly")
dfa2 = dfa1.dropna(how='all')
dfa3 = dfa2.dropna(how='all',axis=1)
#dfa3['Date'] = pd.to_datetime(dfa3['Date'])
dfa4 = dfa3.sort_values(by='Date', ascending = True)
dfa5 = dfa4[['Date','BB Loans', 'B Loans', 'CCC Loans']]

#merge
df7 = pd.merge(df6, dfa5,left_on = 'Date',right_on = 'Date',how = 'left')
df7.columns = ['ELLI- ' + str(col) for col in df7.columns]
df8ELLI = df7.rename(columns={'ELLI- Date': 'Date'})
# df8ELLI.to_excel('//seanas.silvercreeksv.com/shared drive/Investments/Investment Process/Tools/Data Ingestion/LCD/ELLI_GICS_Cleaned.xlsx', index = False)

In [45]:
import numpy as np
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
from math import ceil
from pyodbc import connect
import pandas as pd
connection_string = 'Driver={SQL Server};Server=scdb1.silvercreeksv.com;Database=scfundrisk;Trusted_Connection=yes;'
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
conn = connect(connection_string)
cursor = conn.cursor()
#merge for all

dfAll = pd.merge(df8ELLI, df8LLI,left_on = 'Date', right_on = 'Date', how = 'left')
dfAll.head(10)
# dfAll.to_excel('//seanas.silvercreeksv.com/shared drive/Investments/Investment Process/Tools/Data Ingestion/LCD/All_GICS_Cleaned.xlsx', index = False)

#Backfill
# bf = pd.read_excel('//seanas.silvercreeksv.com/shared drive/Investments/PC Risk/8. Working Files/AB working files/GICSBackfill.xlsx')
# seriesconcat = [dfAll]
# dfAllBackfilled = pd.concat(seriesconcat)
# dfAllBackfilled.to_excel("GICS test 3.xlsx")
FUSEinputLLI = pd.melt(dfAll, id_vars = "Date")


#error handling to account for new proxies, or proxies being taken away
# import pandas as pd
# from sqlalchemy.engine import URL
# from sqlalchemy import create_engine, inspect
# connection_string ='Driver={SQL Server};Server=scdb1.silvercreeksv.com;Database=scfundrisk;Trusted_Connection=yes;'
# connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
# engine = create_engine(connection_url)
# #count how many proxies are coming from the current source to match with what's being pulled
# df = pd.read_sql_query("SELECT * FROM benchmarks WHERE benchmarks.benchmark_type = 'pc-risk' AND benchmarks.benchmark_source = 'lcd'", engine)
# #count number of columns in S&P download subtracting dates column
# cols = len(dfAll.axes[1]) - 1
# if len(df['benchmark_name']) != cols:
# 	raise ValueError("The number of proxies being downloaded does not equal the amount in DB")
# #error handling to account for change in proxy spelling
# #proxies alphabetized from data download
# edf = FUSEinputLLI[['Proxy']]
# edf = edf.drop_duplicates(keep = 'first')
# edf.sort_values('Proxy')
# #proxies alphabetized from DB
# dbdf = df[['benchmark_name']].sort_values('benchmark_name')
# for i in range(len(df)):
#     if edf.iloc[i,0] != dbdf.iloc[i,0]:
#         raise ValueError("The names of the proxies do not match")



#rename columns and drop nulls
FUSEinputLLI.rename(columns = {'variable':'Proxy', 'value':'return_value', 'Date': 'asof_date' }, inplace = True)
FUSEinputLLI = FUSEinputLLI.replace('NA', np.nan)
FUSEinputLLI = FUSEinputLLI.dropna()



#pull in benchmark ID, and replace name with ID for DB upload

lookupid = pd.read_sql_query("select * from benchmarks where benchmark_source = 'lcd'",engine)
FUSEinputLLI = adj_dataframe(FUSEinputLLI)
lookupid = adj_dataframe(lookupid)
dffinale = FUSEinputLLI.merge(lookupid[["benchmark_id","benchmark_name"]], how = "left", left_on = "Proxy", right_on = "benchmark_name", suffixes = (" new", " old"))
dffinale = dffinale[["asof_date", "return_value", "benchmark_id"]]
dffinale['return_value'] = dffinale['return_value'].apply(lambda row: round(row, 15))

#pull in what's already in the database NEED TO CHANGE TO INCLUDE 'BENCHMARK-LCD'
dffinal = pd.read_sql_query("SELECT * FROM benchmark_returns_ts WHERE benchmark_id IN (SELECT benchmark_id from benchmarks WHERE benchmark_source = 'lcd')",engine)

#run function that adjusts column types to make them the same
dffinale = adj_dataframe(dffinale)
dffinal = adj_dataframe(dffinal)

#merge the two
dfinput = dffinale.merge(dffinal, how = "left", on = ["benchmark_id","asof_date"], suffixes = (" new", " old"))
dfinput = adj_dataframe(dfinput)


#WRITE CODE TO CHECK IF NEW = OLD, IF SO THEN SKIP, IF NOT THEN REPLACE

#create a new column checking that new value is equal to the old
dfinput['matching'] = dfinput.apply(lambda x: x['return_value new'] == x['return_value old'], axis=1)

#create a new df with just values that do not match to delete from DB
dfdelete = dfinput[dfinput["matching"] == False]
valstodelete = list[dfdelete['ret_ts_id']] #may want to delete the ret_ts_id
# batch_delete(valstodelete,'benchmark_return_ts', 'ret_ts_id')

dfnewinput = dfinput[dfinput['ret_ts_id'].isnull()]

dfallcombined = pd.concat([dfdelete, dfnewinput])

#clean dfallcombined for same columns as in database


dffinalinput = dfallcombined[["asof_date","return_value new", "source","type"]] #HOW WILL IT KNOW THE SOURCE? IS RET_TS_ID AUTOFILLED?
dffinalinput = dffinalinput.rename(columns = {'return_value new':'return_value'})

#dffinalinput.to_sql('benchmark_returns_ts', engine, if_exists='append', index=False)



# dfinput



In [46]:
dffinalinput

,asof_date,return_value,source,type


In [39]:
dfallcombined

,asof_date,return_value new,benchmark_id,return_value old,source,type,ret_ts_id,matching


In [38]:
dfdelete

,asof_date,return_value new,benchmark_id,return_value old,source,type,ret_ts_id,matching


In [18]:
dfinput

,asof_date,return_value new,benchmark_id,return_value old,source,type,ret_ts_id,matching
0,2020-01-31,0.001852,371,0.001852,lcd,NaN,2415261,True
1,2020-02-29,-0.007770,371,-0.007770,lcd,NaN,2415262,True
2,2020-03-31,-0.184476,371,-0.184476,lcd,NaN,2415263,True
3,2020-04-30,0.122415,371,0.122415,lcd,NaN,2415264,True
4,2020-05-31,0.011719,371,0.011719,lcd,NaN,2415265,True
...,...,...,...,...,...,...,...,...
3523,2022-08-31,0.009380,468,0.009380,lcd,NaN,2443192,True
3524,2022-09-30,-0.023635,468,-0.023635,lcd,NaN,2443193,True
3525,2022-10-31,-0.009448,468,-0.009448,lcd,NaN,2443194,True
3526,2022-11-30,-0.002857,468,-0.002857,lcd,NaN,2443195,True


In [16]:
dfinput

,asof_date,return_value new,benchmark_id,return_value old,source,type,ret_ts_id,matching
0,2020-01-31,0.001852,371,0.001852,lcd,NaN,2415261,True
1,2020-02-29,-0.007770,371,-0.007770,lcd,NaN,2415262,True
2,2020-03-31,-0.184476,371,-0.184476,lcd,NaN,2415263,True
3,2020-04-30,0.122415,371,0.122415,lcd,NaN,2415264,True
4,2020-05-31,0.011719,371,0.011719,lcd,NaN,2415265,True
...,...,...,...,...,...,...,...,...
3489,2022-08-31,0.009380,468,0.009380,lcd,NaN,2443192,True
3490,2022-09-30,-0.023635,468,-0.023635,lcd,NaN,2443193,True
3491,2022-10-31,-0.009448,468,-0.009448,lcd,NaN,2443194,True
3492,2022-11-30,-0.002857,468,-0.002857,lcd,NaN,2443195,True


In [ ]:
# bm_missing_returns.to_sql('benchmark_returns_ts', engine, if_exists='append', index=False)  # index=False prevents failure on trying to insert the index column